In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from core.data_sources import CLOBDataSource

In [ ]:
clob = CLOBDataSource()

CONNECTOR_NAME = "binance"
TRADING_PAIR = "ZEC-USDT"
INTERVAL = "3m"     
DAYS = 15           # days of data to fetch

clob.load_candles_cache()
candles = next(
    (
        c for c in clob.candles_cache.values()
        if c.trading_pair == TRADING_PAIR
        and c.connector_name == CONNECTOR_NAME
        and c.interval == INTERVAL
    ),
    None
)

if not candles:
    candles = await clob.get_candles_last_days(
        CONNECTOR_NAME, TRADING_PAIR, INTERVAL, DAYS, from_trades=False
    )
    clob.dump_candles_cache()

candles_df = candles.data


In [ ]:
macd_fast = 12
macd_slow = 26
macd_signal = 9
rsi_period = 14
momentum_target = 0.005  # 0.5% exit target

In [ ]:
macd_df = ta.macd(candles_df["close"], fast=macd_fast, slow=macd_slow, signal=macd_signal)
# RSI
rsi = ta.rsi(candles_df["close"], length=rsi_period)

candles_df = candles_df.join(macd_df)
candles_df["RSI_14"] = rsi

In [ ]:
candles_df["buy_signal"] = (
    (candles_df["MACD_12_26_9"] > candles_df["MACDs_12_26_9"])  # MACD crosses above Signal
    & (candles_df["MACD_12_26_9"].shift(1) <= candles_df["MACDs_12_26_9"].shift(1))
    & (candles_df["MACDh_12_26_9"] > candles_df["MACDh_12_26_9"].shift(1))  # Histogram expanding positive
    & (candles_df["RSI_14"] > 50)
)

candles_df["sell_signal"] = (
    (candles_df["MACD_12_26_9"] < candles_df["MACDs_12_26_9"])  # MACD crosses below Signal
    & (candles_df["MACD_12_26_9"].shift(1) >= candles_df["MACDs_12_26_9"].shift(1))
    & (candles_df["MACDh_12_26_9"] < candles_df["MACDh_12_26_9"].shift(1))  # Histogram expanding negative
    & (candles_df["RSI_14"] < 50)
)

# Optional: Exit when MACD crosses zero (momentum fade)
candles_df["exit_signal"] = (
    (candles_df["MACD_12_26_9"] * candles_df["MACD_12_26_9"].shift(1) < 0)
)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=candles_df["timestamp"],
    open=candles_df["open"],
    high=candles_df["high"],
    low=candles_df["low"],
    close=candles_df["close"],
    name="Price",
    increasing_line_color="#26A69A",
    decreasing_line_color="#EF5350"
))

# --- Buy and Sell Markers
fig.add_trace(go.Scatter(
    x=candles_df.loc[candles_df["buy_signal"], "timestamp"],
    y=candles_df.loc[candles_df["buy_signal"], "close"],
    mode="markers",
    name="Buy Signal",
    marker=dict(symbol="triangle-up", color="#00FF00", size=10)
))
fig.add_trace(go.Scatter(
    x=candles_df.loc[candles_df["sell_signal"], "timestamp"],
    y=candles_df.loc[candles_df["sell_signal"], "close"],
    mode="markers",
    name="Sell Signal",
    marker=dict(symbol="triangle-down", color="#FF0000", size=10)
))
fig.add_trace(go.Scatter(
    x=candles_df.loc[candles_df["exit_signal"], "timestamp"],
    y=candles_df.loc[candles_df["exit_signal"], "close"],
    mode="markers",
    name="Exit Signal",
    marker=dict(symbol="x", color="#FFFF00", size=9)
))

# --- MACD, Signal, Histogram (secondary axis)
fig.add_trace(go.Scatter(
    x=candles_df["timestamp"],
    y=candles_df["MACD_12_26_9"],
    name="MACD Line",
    line=dict(color="#2196F3", width=1.5),
    yaxis="y2"
))
fig.add_trace(go.Scatter(
    x=candles_df["timestamp"],
    y=candles_df["MACDs_12_26_9"],
    name="Signal Line",
    line=dict(color="#FFB300", width=1.2),
    yaxis="y2"
))
fig.add_trace(go.Bar(
    x=candles_df["timestamp"],
    y=candles_df["MACDh_12_26_9"],
    name="Histogram",
    marker_color=np.where(candles_df["MACDh_12_26_9"] >= 0, "#26A69A", "#EF5350"),
    yaxis="y2",
    opacity=0.5
))

# --- RSI subplot
fig.add_trace(go.Scatter(
    x=candles_df["timestamp"],
    y=candles_df["RSI_14"],
    name="RSI(14)",
    line=dict(color="#9C27B0", width=1.2),
    yaxis="y3"
))


In [ ]:
fig.update_layout(
    title="MACD Momentum Cross Strategy — BTC/USDT",
    template="plotly_dark",
    height=900,
    xaxis=dict(domain=[0, 1], rangeslider=dict(visible=False)),
    yaxis=dict(title="Price"),
    yaxis2=dict(title="MACD", overlaying="y", side="right", showgrid=False),
    yaxis3=dict(title="RSI", anchor="free", overlaying="y", side="right", position=1.08),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.show()

### MACD Momentum Strategy

Implemented a MACD Momentum Crossover strategy. The system uses the MACD line and signal line crossovers to capture trend momentum, filtered by RSI to improve entry accuracy. It aims to enter trades in the direction of increasing momentum and exit when momentum weakens.

## Features:

1. MACD (12,26,9) for detecting momentum shifts and trend strength

2. RSI (14) for filtering trades (longs when RSI > 50, shorts when RSI < 50)

3. Entry conditions — buy when the MACD line crosses above the signal line with a rising histogram; sell on the opposite cross

4. Exit rules — exit at zero-line cross or on a fixed profit target (0.5–1%)

Check the controller file `macd_momentum.py` in `app\controllers\directional_trading` for controller implementation